# Part 1: Getting Started with CRIPT (A Community Resource for Innovation in Polymer Technology)

**Welcome to CRIPT!**
CRIPT's mission is to develop a digital ecosystem for polymers. As part of that mission, we have generated a data model which provides rules on how to format polymer data so that it can be easily stored, shared, and searched. This tutorial will introduce you to Python bindings for the CRIPT API. Once CRIPT is available via PyPI, you will no longer need to clone a local version of CRIPT Python. 


## Connecting to the CRIPT platform
To establish a connection to the CRIPT platform we will utilize the CRIPT API. You should first create an account on the CRIPT platform and retrive an API token.

## CRIPT Nodes
CRIPT nodes are objects on the CRIPT platform that are used to store information and data (e.g. `User`, `Group`, `Collection`, `Experiment`). Nodes have a dependency structure that is outlined by the CRIPT data model (e.g. a `Condition` node must be linked to a `Property` node)).

### Primary Nodes
Primary nodes can be accessed via the root level API (https://criptapp.herokuapp.com/api/). Each primary node will have an associated URL. 

* `User`
    * Contains information about an individual using the database (e.g. John Doe, Jane Doe).
* `Group`
    * A foundational node for storing information about a research team. All other nodes must reference a `Group` (e.g. CRIPT development team, Olsen lab).
* `Collection`
    * Contains a grouping of experiments, with the purpose to provide a hierarchical organization to non-linear, but related experiments (e.g. ROMP Kinetic Experiments, Polyolefin Library).
* `Experiment`
    * A set of conditions and performed processes that result in a material (e.g. Synthesis of PS-b-PB, ATRP of styrene).
* `Inventory`
    * Well defined materials typically used in an `Experiment` (e.g. Olsen Lab Inventory, ROP monomers, Stockroom Chemicals)
* `Material`
    * Indentifying information and property data for a chemical (e.g. styrene, toluene, polystyrene, polyethylene).
* `Identity`
* `Process`
    * A procedural set of steps in an `Experiment` (e.g. Anionic Polymerization of styrene)
* `Step`
    * A defined piece of a `Process`.
* `Data`
    * Contains metadata and data associated with an experiment (e.g. 1H NMR, SEC trace, IR spectrum)
* `File`
    * Contains metadata and link to a data file stored on CRIPT.
* `Reference`
    * 

### Secondary nodes
- `Citation`
- `Component`
- `Condition`
* `IntermediateIngredient`
    * Material ingredients that are generated as part of an experiment but not a final product.
* `MaterialIngredient`
    * Adds quantity field to Material (e.g. 10g of Styrene).
- `Property`
- `Quantity`

In [2]:
import os

# this assumes you have downloaded the CRIPT python package
import sys
sys.path.append('../../')
from cript import *

# Initialize API session (i.e. login)
# You will need to get your API token from the account settings page after logging in (https://criptapp.herokuapp.com/settings/)
# We recommend storing your API token as an evironment variable
# Once connected, you should be able to view the API easily here: https://criptapp.herokuapp.com/api/

session = connect.API(url="https://criptapp.herokuapp.com", token=os.environ['CRIPT_API_TOKEN'])
session


Connection to the API was successful!



Connected to https://criptapp.herokuapp.com/api

In [3]:
# running a query without defining any parameters will return all nodes available to that user.
# Search queries can only be made on primary nodes
query_params = {'':''}
query = session.search(Experiment, query_params)
print(query)

{'count': 1, 'next': None, 'previous': None, 'results': [{'url': 'https://criptapp.herokuapp.com/api/experiment/1/', 'name': 'Anionic Polymerization of Styrene with SecBuLi', 'funding': None, 'notes': None, 'collection': 'https://criptapp.herokuapp.com/api/collection/1/', 'processes': [], 'data': ['https://criptapp.herokuapp.com/api/data/1/', 'https://criptapp.herokuapp.com/api/data/2/'], 'created_at': '2022-03-15T04:31:30.424911Z', 'updated_at': '2022-03-15T19:00:20.192142Z', 'group': 'https://criptapp.herokuapp.com/api/group/1/', 'public': True}]}


Query fields for primary keys:
* `User`: email, url
* `Group`: name, users, url
* `Collection`: group, name, public, url, notes, experiments, inventories, citations
* `Experiment`: group, name, public, url, collection, funding, notes, processes, data,
* `Inventory`: group, name, public, url, collection, description
* `Material`: group, name, public, url, components, vendor, lot_number, keywords, notes, step, properties, citations
* `Identity`: group, name, public, url, names, cas, smiles, bigsmiles, chem_formula, chem_repeat, pubchem_cid, inchi, inchi_key
* `Process`: group, name, public, url, experiments, keywords, notes, steps
* `Step`: group, public, url, process, step_id, type, description, intermediate_ingredients, material_ingredients, equipment, duration, time_position, properties, conditions, set_id, material_products, dependent_steps
* `Data`: group, name, public, url, files, type, sample_prep, calibration, configuration, notes, experiments, materials, steps, citations
* `File`: group, name, public, url, data, type, id, source, extension, external_source
* `Reference`: group, public, url, title, doi, authors, journal, publisher, year, volume, pages, issn, arxiv_id, mpid, website, notes

Types:
* Name = String 

In [4]:
query_params = {'name':'Citrine'}
query = session.search(Group, query_params)
print(query)

{'count': 1, 'next': None, 'previous': None, 'results': [{'url': 'https://criptapp.herokuapp.com/api/group/7/', 'name': 'Citrine', 'users': ['https://criptapp.herokuapp.com/api/user/9/']}]}


In [ ]:
# create and register a collection node to the database
collection_node = nodes.Collection(name='cborg_collection', group='https://criptapp.herokuapp.com/api/group/7/')
session.save(collection_node)